In [4]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
from sklearn.metrics import confusion_matrix
from itertools import *
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('fivethirtyeight')
import os
import shutil
import random
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  0


## Loading Data

Data was obtained from a [Kaggle competition](https://www.kaggle.com/c/dogs-vs-cats/data).

Let's organize the data into:
- Training Data
- Validation Data
- Testing Data

In [12]:
!ls

all_images


In [13]:
# change the directory to the dogs and cats directories
# os.chdir('data/dogs-cats-data/')
# If the the train/dogs diretory does not exist then proceed to create some directories
if os.path.isdir('train/dog') is False:
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')
    
    for e in random.sample(glob.glob('all_images/cat*'), 500):
        shutil.move(e,'train/cat')
    for e in random.sample(glob.glob('all_images/dog*'), 500):
        shutil.move(e,'train/dog')
    for e in random.sample(glob.glob('all_images/cat*'), 100):
        shutil.move(e,'valid/cat')
    for e in random.sample(glob.glob('all_images/dog*'), 100):
        shutil.move(e,'valid/dog')
    for e in random.sample(glob.glob('all_images/cat*'), 50):
        shutil.move(e,'test/cat')
    for e in random.sample(glob.glob('all_images/dog*'), 50):
        shutil.move(e,'test/dog')

In [15]:
# os.chdir('../../')        

In [17]:
!ls

convolution_neural_animals.ipynb data


In [18]:
train_path = 'data/dogs-cats-data/train/'
valid_path = 'data/dogs-cats-data/valid/'
test_path = 'data/dogs-cats-data/test/'

In [23]:
# The images are passsed to the prerpocessing function before they get passed to the network
# target_size: it is the height and width of what we want the images to be resized to


train_batches = ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory = train_path, target_size = (224,224), classes = ['cat', 'dog'], batch_size = 10)
valid_batches = ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory = valid_path, target_size = (224,224), classes = ['cat', 'dog'], batch_size = 10)
test_batches = ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory = test_path, target_size = (224,224), classes = ['cat', 'dog'], batch_size = 10, shuffle = False)


Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [27]:
assert train_batches.n == 1000
assert valid_batches.n == 200
assert test_batches.n == 100
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 2

In [28]:
imgs, labels = next(train_batches)

In [29]:
def plotImages(images_arr):
    """
    A function that plots images in the form of a grid with 1 row and 10 columns where images
    are placed
    """
    fig, axes = plt.subplots(1,10, figsize = (20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()